In [1]:
import numpy as np
# from manipulation_master.manipulation.utils import running_as_notebook
# from manipulation_master.manipulation.meshcat_utils import AddMeshcatTriad
# from manipulation_master.manipulation.scenarios import AddMultibodyTriad
from pydrake.all import (ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant, Parser,
                         PiecewisePolynomial, PiecewiseQuaternionSlerp,
                         RigidTransform, RollPitchYaw, RotationMatrix,
                         Simulator, Solve, StartMeshcat, TrajectorySource)
                         
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

import os
import sys
import time
from collections import namedtuple
from functools import partial

import numpy as np
from IPython.display import HTML, Javascript, display
from pydrake.common.value import AbstractValue
from pydrake.geometry import (Cylinder, MeshcatVisualizer,
                              MeshcatVisualizerParams, Rgba, Role, Sphere)
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import BodyIndex, JointIndex
from pydrake.perception import BaseField, Fields, PointCloud
from pydrake.solvers.mathematicalprogram import BoundingBoxConstraint
from pydrake.systems.framework import (DiagramBuilder, EventStatus, LeafSystem,
                                       PublishEvent, VectorSystem)

from meshcat_utils import MeshcatPoseSliders, WsgButtonPanda, PandaHandButton

from station import MakeChessManipulationStation, AddPandaDifferentialIK


In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [4]:
builder = DiagramBuilder()
station = builder.AddSystem(
    MakeChessManipulationStation())
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")
controller_plant = station.GetSubsystemByName(
    "panda_controller").get_multibody_plant_for_control()

viz = MeshcatVisualizer.AddToBuilder(
    builder,
    station.GetOutputPort("query_object"),
    meshcat)

meshcat.ResetRenderMode()
meshcat.DeleteAddedControls()

# Set up differential inverse kinematics.
differential_ik = AddPandaDifferentialIK( 
    builder,
    controller_plant,
    frame=controller_plant.GetFrameByName("panda_link8"))
builder.Connect(differential_ik.get_output_port(),
                station.GetInputPort("panda_position"))
builder.Connect(station.GetOutputPort("panda_state_estimated"),
                differential_ik.GetInputPort("robot_state"))

# Set up teleop widgets.
q0 = [0.0, 0, 0.0, -np.pi/2, 0.0, np.pi/2, np.pi/4]
teleop = builder.AddSystem(
    MeshcatPoseSliders(
        meshcat,
        min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                pitch=-0.5,
                                                yaw=-np.pi,
                                                x=-0.4,
                                                y=-0.25,
                                                z=0.0),
        max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                pitch=np.pi,
                                                yaw=np.pi,
                                                x=0.4,
                                                y=0.25,
                                                z=0.75),
        body_index=plant.GetBodyByName("panda_link8").index(),
        # It seems that value is set by the default joint positions, not here.
        value=MeshcatPoseSliders.Value(roll=0.0,  # idk if this part works...  
                                    pitch=0.0,
                                    yaw=0.0,
                                    x=0.0,
                                    y=0.0,
                                    z=0.5)))
builder.Connect(teleop.get_output_port(0),
                differential_ik.get_input_port(0))
builder.Connect(station.GetOutputPort("body_poses"),
                teleop.GetInputPort("body_poses"))

wsg_teleop = builder.AddSystem(WsgButtonPanda(meshcat))
builder.Connect(wsg_teleop.get_output_port(0),
                station.GetInputPort("Schunk_Gripper_position"))
builder.Connect(station.GetOutputPort("Schunk_Gripper_state_measured"),
                wsg_teleop.GetInputPort('wsg_state_measured'))
# builder.Connect(station.GetOutputPort("Schunk_Gripper_force_measured"),
#                 wsg_teleop.GetInputPort('wsg_force_measured'))

diagram = builder.Build()

# plant.SetPositions(context_plant, plant.GetModelInstanceByName("panda"), q0)

simulator = Simulator(diagram)
context = simulator.get_mutable_context()

simulator.set_target_realtime_rate(1.0)

meshcat.AddButton("Stop Simulation", "Escape")
print("Press Escape to stop the simulation")
while meshcat.GetButtonClicks("Stop Simulation") < 1:
    simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
meshcat.DeleteButton("Stop Simulation")


name:  WorldModelInstance
name:  DefaultModelInstance
name:  panda
name:  Schunk_Gripper
name:  Board
name:  a1RW
name:  b1NW
name:  c1BW
name:  d1QW
name:  e1KW
name:  f1BW
name:  g1NW
name:  h1RW
name:  a2PW
name:  b2PW
name:  c2PW
name:  d2PW
name:  e2PW
name:  f2PW
name:  g2PW
name:  h2PW
name:  a8RB
name:  b8NB
name:  c8BB
name:  d8QB
name:  e8KB
name:  f8BB
name:  g8NB
name:  h8RB
name:  a7PB
name:  b7PB
name:  c7PB
name:  d7PB
name:  e7PB
name:  f7PB
name:  g7PB
name:  h7PB
<class 'meshcat_utils.Value'>
Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
Press Escape to stop the simulation


RuntimeError: MultibodyPlant's discrete update solver failed to converge at simulation time = 77.102 with discrete update period = 0.002. This usually means that the plant's discrete update period is too large to resolve the system's dynamics for the given simulation conditions. This is often the case during abrupt collisions or during complex and fast changing contact configurations. Another common cause is the use of high gains in the simulation of closed loop systems. These might cause numerical instabilities given our discrete solver uses an explicit treatment of actuation inputs. Possible solutions include:
  1. reduce the discrete update period set at construction,
  2. decrease the high gains in your controller whenever possible,
  3. switch to a continuous model (discrete update period is zero),      though this might affect the simulation run time.

In [ ]:
# OLD

scene_graph = station.GetSubsystemByName("scene_graph")

viz = MeshcatVisualizer.AddToBuilder(
    builder,
    station.GetOutputPort("query_object"),
    meshcat)

q0 = [0.0, 0, 0.0, -np.pi/2, 0.0, np.pi/2, np.pi/4, 0.04, 0.04]
t_lst = np.linspace(0, 5, 30)
# q_knots = np.repeat(np.array(q0).reshape(1, -1), 30, axis=0)
q_knots = []
working_q = np.array(q0)
dt = 0.01
for i in range(30):
    working_q[0] += dt
    q_knots.append(working_q.copy())

print(q_knots)
q_knots = np.vstack(q_knots)
print(q_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:9].T)

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))

builder.Connect(q_traj_system.get_output_port(),
                station.GetInputPort("panda_position"))

diagram = builder.Build()

context_diagram = diagram.CreateDefaultContext()
context_station = diagram.GetSubsystemContext(station, context_diagram)
context_scene_graph = station.GetSubsystemContext(scene_graph, context_station)
context_plant = station.GetMutableSubsystemContext(plant, context_station)

plant.SetPositions(context_plant, plant.GetModelInstanceByName("panda"), q0)

diagram.Publish(context_diagram)

simulator = Simulator(diagram)
simulator.set_publish_at_initialization(True)
simulator.Initialize()
simulator.set_target_realtime_rate(1.0)

simulator.AdvanceTo(5)






In [ ]:
# THIS CELL NOT USED
builder = DiagramBuilder()
station = builder.AddSystem(
    MakeChessManipulationStation())
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

viz = MeshcatVisualizer.AddToBuilder(
    builder,
    station.GetOutputPort("query_object"),
    meshcat)

q0 = [0.0, 0, 0.0, -np.pi/2, 0.0, np.pi/2, np.pi/4, 0.04, 0.04]
t_lst = np.linspace(0, 5, 30)
# q_knots = np.repeat(np.array(q0).reshape(1, -1), 30, axis=0)
q_knots = []
working_q = np.array(q0)
dt = 0.01
for i in range(30):
    working_q[0] += dt
    q_knots.append(working_q.copy())

print(q_knots)
q_knots = np.vstack(q_knots)
print(q_knots)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:9].T)

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))

builder.Connect(q_traj_system.get_output_port(),
                station.GetInputPort("panda_position"))

diagram = builder.Build()

context_diagram = diagram.CreateDefaultContext()
context_station = diagram.GetSubsystemContext(station, context_diagram)
context_scene_graph = station.GetSubsystemContext(scene_graph, context_station)
context_plant = station.GetMutableSubsystemContext(plant, context_station)

plant.SetPositions(context_plant, plant.GetModelInstanceByName("panda"), q0)

diagram.Publish(context_diagram)

simulator = Simulator(diagram)
simulator.set_publish_at_initialization(True)
simulator.Initialize()
simulator.set_target_realtime_rate(1.0)

simulator.AdvanceTo(5)




